<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/tirgul-Omer-Matan/Lab7_Omer_Matan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Document Service
class DocumentService:
    def __init__(self):
        self.documents = {}
        self.next_id = 1

    def add_document(self, title, content):
        doc_id = str(self.next_id)
        self.documents[doc_id] = {
            "id": doc_id,
            "title": title,
            "content": content
        }
        self.next_id += 1
        return self.documents[doc_id]

    def get_document(self, doc_id):
        return self.documents.get(doc_id)

# Index Service
import re
from collections import defaultdict

class IndexService:
    def __init__(self):
        self.index = defaultdict(set)

    def index_document(self, doc_id, content):
        words = re.findall(r"[a-zA-Z]+", content.lower())
        for word in words:
            self.index[word].add(doc_id)

    def search(self, terms, mode="AND"):
        if not terms:
            return set()
        sets = [self.index.get(term.lower(), set()) for term in terms]
        return set.intersection(*sets) if mode == "AND" else set.union(*sets)

# Rating Service
class RatingService:
    def __init__(self):
        self.ratings = defaultdict(list)  # doc_id -> list of scores

    def add_rating(self, doc_id, score):
        self.ratings[doc_id].append(score)

    def get_avg_rating(self, doc_id):
        scores = self.ratings.get(doc_id, [])
        return round(sum(scores) / len(scores), 2) if scores else None

# Result Formatter (Optional Service)
def format_results(doc_ids, doc_service, rating_service):
    results = []
    for doc_id in doc_ids:
        doc = doc_service.get_document(doc_id)
        if doc:
            results.append({
                "doc_id": doc_id,
                "title": doc["title"],
                "snippet": doc["content"][:100] + "...",
                "score": rating_service.get_avg_rating(doc_id)
            })
    return results


# Simulation:
doc_service = DocumentService()
index_service = IndexService()
rating_service = RatingService()

# Add documents
d1 = doc_service.add_document("AI Article", "Artificial intelligence is the future of technology.")
d2 = doc_service.add_document("Cloud Computing", "Cloud computing enables scalable architectures.")
d3 = doc_service.add_document("Intro to Python", "Python is a programming language used in AI and cloud.")

# Index them
for doc in [d1, d2, d3]:
    index_service.index_document(doc["id"], doc["content"])

# Add ratings
rating_service.add_rating("1", 5)
rating_service.add_rating("1", 4)
rating_service.add_rating("2", 3)
rating_service.add_rating("3", 4)

# Search with AND
results_and = index_service.search(["ai", "cloud"], mode="AND")

# Search with OR
results_or = index_service.search(["ai", "cloud"], mode="OR")

# Display
print("Search Results (AND):", format_results(results_and, doc_service, rating_service))
print("Search Results (OR):", format_results(results_or, doc_service, rating_service))

Search Results (AND): [{'doc_id': '3', 'title': 'Intro to Python', 'snippet': 'Python is a programming language used in AI and cloud....', 'score': 4.0}]
Search Results (OR): [{'doc_id': '3', 'title': 'Intro to Python', 'snippet': 'Python is a programming language used in AI and cloud....', 'score': 4.0}, {'doc_id': '2', 'title': 'Cloud Computing', 'snippet': 'Cloud computing enables scalable architectures....', 'score': 3.0}]
